In [ ]:
# produciamo una predizione sulla base dei dati ricevuti da Max/MSP e ne inviamo il risultato a Max/MSP
# USARE qs modello: Boids_225_10.h5

from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import joblib

import socket

from pythonosc.udp_client import SimpleUDPClient
from pythonosc import osc_message_builder
from pythonosc.dispatcher import Dispatcher
from pythonosc.osc_server import BlockingOSCUDPServer

import matplotlib.pyplot as plt

port1 = 7002
port2 = 6002
ip = "127.0.0.1"
dim_dati = 9
predictions = []
values = []
data_buffer = []
x_vals = []
y_vals = []


def osc_handler(address, *args):
    """Funzione che gestisce i dati ricevuti."""
    # importiamo lo scalaggio introdotto dalla normalizzazione pre-training per 
    # calcolare la de-normalizzazione post-predizione
    y_scaler = joblib.load("y_scaler.pkl")
    
    global values, data_buffer, x_vals, y_vals
    
    if len(args) >= 2:
        x, y = float(args[0]), float(args[1])
        values.append(x)
        values.append(y)
        data_buffer.append(values)
        print("data_buffer ", data_buffer)
        values=[]
        print(f"✅ Ricevuto da Max/MSP: x = {x}, y = {y}")
        ctrl_data = np.array(data_buffer)  # forma (50, 6) # Se il tuo modello si aspetta un batch di shape (N, 6) va bene così,
        predictions = model.predict(ctrl_data)
        # NON li riscalo. Tengo le predizioni normalizzate, ossia tra 0 e 1.
        #print("predictions", predictions)
        #y_pred_scaled = np.array(predictions)
        #predictions_original = y_scaler.inverse_transform(y_pred_scaled )
        #print("predictions_original: ", predictions_original)

        client = SimpleUDPClient(ip, port2)  # Crea un client

        for ind in range(predictions.shape[1]): # 9
            #x_val = predictions_original[0,ind]
            x_val = predictions[0,ind]
            #print("predictions[ind]",predictions[ind])
            print("x_val",x_val)
            prefix_x = f"/x{int(ind)}"
            print("prefix_x", prefix_x)
            client.send_message(prefix_x, float(x_val))

            x_vals.append(x_val)
     
    # Svuoto il buffer per il prossimo batch
    data_buffer.clear()

# Carica il modello Keras salvato in precedenza
modello = "Synth_model_2x6_9.h5" # funziona BENE!
model = load_model(modello)
print("Modello", modello, "caricato correttamente.")


# Avvia l'ascolto dei dati
dispatcher = Dispatcher()
dispatcher.map("/*", osc_handler)  # Assegna la funzione a tutti i messaggi OSC

# Configura il server OSC
UDP_IP = "127.0.0.1"  # Indirizzo locale
UDP_PORT = port1       # Porta su cui Max/MSP sta inviando

server = BlockingOSCUDPServer((UDP_IP, UDP_PORT), dispatcher)
print(f"🔵 Server OSC in ascolto su {UDP_IP}:{UDP_PORT}...")

# Avvia il server OSC
server.serve_forever()


# Avvia l'ascolto dei dati
#listen_ctrl_data(port=port1, batch_size=50)


Modello Synth_model_2x6_27.h5 caricato correttamente.
🔵 Server OSC in ascolto su 127.0.0.1:7002...
data_buffer  [[0.07086614519357681, 0.0]]
✅ Ricevuto da Max/MSP: x = 0.07086614519357681, y = 0.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
x_val -8.915335
prefix_x /x0
x_val 2.2167804
prefix_x /x1
x_val 7.598527
prefix_x /x2
x_val 3.5905406
prefix_x /x3
x_val 2913.2573
prefix_x /x4
x_val -5.4399447
prefix_x /x5
